In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
import random

import torch
import torch.nn as nn
import torch.nn.functional as F



In [11]:
import sys
print(sys.executable)

!pip show pytorch-tabnet

C:\Users\smhrd\anaconda3\python.exe
Name: pytorch-tabnet
Version: 4.1.0
Summary: PyTorch implementation of TabNet
Home-page: https://github.com/dreamquark-ai/tabnet
Author: 
Author-email: 
License: 
Location: C:\Users\smhrd\anaconda3\Lib\site-packages
Requires: numpy, scikit_learn, scipy, torch, tqdm
Required-by: 


In [9]:
import sys
!{sys.executable} -m pip install pytorch-tabnet

  Using cached pytorch_tabnet-4.1.0-py3-none-any.whl.metadata (15 kB)
Using cached pytorch_tabnet-4.1.0-py3-none-any.whl (44 kB)


In [15]:
import random

# 전체 시드 고정 코드 (한 번만 실행하면 됨)
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # GPU에서도 고정
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [10]:
# ✅ 딥러닝 모델 3종 성능 비교 (PyTorch MLP, TabNet, FT-Transformer)



# ✅ 1. PyTorch 기반 MLP 모델 정의
class FeatureWeightedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64], dropout_rate=0.5):
        super().__init__()
        self.layers = nn.ModuleList()
        prev_dim = input_dim
        for dim in hidden_dims:
            self.layers.append(nn.Linear(prev_dim, dim))  # 각 hidden layer 추가
            prev_dim = dim
        self.output_layer = nn.Linear(prev_dim, 1)  # 출력층
        self.dropout_rate = dropout_rate

    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))  # 활성화 함수 적용
            if self.dropout_rate > 0:
                x = F.dropout(x, p=self.dropout_rate, training=self.training)  # 드롭아웃 적용
        x = torch.sigmoid(self.output_layer(x))  # 이진 분류를 위한 시그모이드 출력
        return x


# ✅ 1-1. MLP 학습 및 교차검증 평가 함수

def train_evaluate_pytorch_mlp(X, y, folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    roc_list, pr_list = [], []

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # 정규화
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        # 텐서 변환
        X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
        X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
        y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).to(device)

        model = FeatureWeightedMLP(input_dim=X.shape[1]).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.BCELoss()  # 이진 분류용 loss

        # 에폭 반복
        for epoch in range(100):
            model.train()
            optimizer.zero_grad()
            preds = model(X_train_tensor).squeeze()
            loss = criterion(preds, y_train_tensor)
            loss.backward()
            optimizer.step()

        # 검증
        model.eval()
        with torch.no_grad():
            preds = model(X_val_tensor).squeeze().cpu().numpy()
        roc_list.append(roc_auc_score(y_val, preds))
        pr_list.append(average_precision_score(y_val, preds))

    return np.mean(roc_list), np.mean(pr_list)


# ✅ 2. TabNet 모델 학습 및 평가

def train_evaluate_tabnet(X, y, folds=5):
    X_np = X.values
    y_np = y.values.reshape(-1, 1)
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    roc_list, pr_list = [], []

    for train_idx, val_idx in skf.split(X_np, y_np):
        X_train, X_val = X_np[train_idx], X_np[val_idx]
        y_train, y_val = y_np[train_idx], y_np[val_idx]
        y_train, y_val = y_train.reshape(-1), y_val.reshape(-1)
        model = TabNetClassifier(verbose=0)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric=['auc'],
                  max_epochs=100, patience=20)

        preds = model.predict_proba(X_val)[:, 1]  # 클래스 1의 확률
        roc_list.append(roc_auc_score(y_val, preds))
        pr_list.append(average_precision_score(y_val, preds))

    return np.mean(roc_list), np.mean(pr_list)


# ✅ 3. FT-Transformer (임시 대체: LogisticRegression 사용)
def train_evaluate_ft_transformer(X, y, folds=5):
    from sklearn.linear_model import LogisticRegression  # 실제 FTTransformer 대체용
    roc_list, pr_list = [], []
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        clf = LogisticRegression(max_iter=1000)  # 단순 선형 모델로 대체
        clf.fit(X_train_scaled, y_train)
        preds = clf.predict_proba(X_val_scaled)[:, 1]

        roc_list.append(roc_auc_score(y_val, preds))
        pr_list.append(average_precision_score(y_val, preds))

    return np.mean(roc_list), np.mean(pr_list)


# ✅ 4. 통합 성능 비교 함수

def compare_all_models(X, y):
    print("\n📊 Comparing PyTorch MLP...")
    mlp_roc, mlp_pr = train_evaluate_pytorch_mlp(X, y)
    print(f"MLP → ROC AUC: {mlp_roc:.4f}, PR AUC: {mlp_pr:.4f}")

    print("\n📊 Comparing TabNet (PyTorch)...")
    tabnet_roc, tabnet_pr = train_evaluate_tabnet(X, y)
    print(f"TabNet → ROC AUC: {tabnet_roc:.4f}, PR AUC: {tabnet_pr:.4f}")

    print("\n📊 Comparing FT-Transformer (Simulated)...")
    ft_roc, ft_pr = train_evaluate_ft_transformer(X, y)
    print(f"FT-Transformer → ROC AUC: {ft_roc:.4f}, PR AUC: {ft_pr:.4f}")

    # 최종 결과 요약 출력
    print("\n✅ Summary")
    print(f"PyTorch MLP     : ROC AUC = {mlp_roc:.4f}, PR AUC = {mlp_pr:.4f}")
    print(f"TabNet          : ROC AUC = {tabnet_roc:.4f}, PR AUC = {tabnet_pr:.4f}")
    print(f"FT-Transformer  : ROC AUC = {ft_roc:.4f}, PR AUC = {ft_pr:.4f}")


SystemExit: usage: ipykernel_launcher.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: ipykernel_launcher.py --help [cmd1 cmd2 ...]
   or: ipykernel_launcher.py --help-commands
   or: ipykernel_launcher.py cmd --help

error: option -f not recognized

In [4]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('data/(최종)_서울열선_광진도로.csv')
X = pd.get_dummies(data[['도로 종류', '도로폭', '경사각', '최근접_시설의_평균거리', '종합_평균_기온', '생활인구', '최근접_시설들_최소거리', '최근접_시설들_최대거리']])
y = data['열선']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train = X_train.astype('float')
X_test = X_test.astype('float')

compare_all_models(X_train, y_train)

NameError: name 'pd' is not defined

In [19]:
X_train.values.shape

(5727, 15)

In [85]:
import torch.optim as optim

# 1. 더미 데이터 생성 (10개 샘플, 3개 특성)
X = torch.randn(10, 3)
y = torch.randint(0, 2, (10,)).float()

# 2. 단순한 MLP 모델 (입력 3 → 은닉층 5 → 출력 1)
model = nn.Sequential(
    nn.Linear(3, 5),
    nn.ReLU(),
    nn.Linear(5, 1),
    nn.Sigmoid()
)

# 3. 손실 함수와 옵티마이저 정의
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 4. Forward pass: 예측
y_pred = model(X).squeeze()

# 5. 손실 계산
loss = criterion(y_pred, y)
print(f"Loss: {loss.item():.4f}")

# 6. 역전파: gradient 계산
loss.backward()

# 7. 각 레이어의 gradient 출력 (before update)
print("\nGradients before update:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} grad:\n{param.grad}")

# 8. 가중치 업데이트
optimizer.step()

# 9. 업데이트 후 가중치 확인
print("\nWeights after update:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}:\n{param.data}")

Loss: 0.8005

Gradients before update:
0.weight grad:
tensor([[ 0.0261, -0.0154,  0.0210],
        [ 0.0363, -0.0241,  0.0322],
        [-0.0589, -0.0133, -0.0261],
        [-0.0453,  0.1042, -0.0556],
        [ 0.0041, -0.0320,  0.0046]])
0.bias grad:
tensor([ 0.0090,  0.0168,  0.0425, -0.0613,  0.0153])
2.weight grad:
tensor([[-0.0022, -0.0223, -0.1134, -0.0328, -0.1020]])
2.bias grad:
tensor([-0.3158])

Weights after update:
0.weight:
tensor([[ 0.5429,  0.1643, -0.3049],
        [ 0.4136, -0.1097, -0.3877],
        [-0.4631, -0.2633,  0.2436],
        [ 0.1357, -0.1871, -0.1949],
        [-0.1461, -0.3870,  0.5336]])
0.bias:
tensor([-0.5135, -0.0686,  0.0749,  0.0087, -0.0268])
2.weight:
tensor([[-0.3546, -0.1918, -0.2372,  0.4107, -0.1262]])
2.bias:
tensor([-0.2895])
